Just look to see if the sequences overlap

In [ ]:
## Load any changes to local modules
%load_ext autoreload
%autoreload 2

import os
import sys

pwd = %pwd
project_dir = '{0}/../../../'.format(pwd)
module_path = os.path.abspath(os.path.join(project_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import re
import numpy as np
%matplotlib inline
from IPython.display import display


#from nbcpact import AnalyzeQuantCompare,Peptide,PeptideGroup,UcbreUtils,PeptidesFromPeptideListBuilder
from nbcpact.protdisc import PDReader
from nbcpact.ucbre import AnalyzeQuantCompare, PeptidesFromPeptideListBuilder

In [ ]:
peptide_list_file = '/home/jonesmic/gBuild/jonesmic_github/proteomics-scripts/datanocommit/peptideList.csv'
peptidesFromPeptideListBuilder = PeptidesFromPeptideListBuilder(peptide_list_file)
analyzeQuantCompare = AnalyzeQuantCompare(peptide_generator=peptidesFromPeptideListBuilder)
groups = analyzeQuantCompare.build_peptide_groups()
results = analyzeQuantCompare.build_results_from_peptide_groups(groups=groups)

In [ ]:
def load_ip2_peptide_list(file_path):
    
    df = pd.read_csv(file_path)
    df['ip2_sequence'] = df.sequence

    df['sequence'] = df.sequence.str.replace('\(\d+\.\d+\)','')

    num_records = df.index.size
    num_seqs = len(set(df['sequence']))

    #ip2_peptide_listDF = ip2_peptide_listDF[~ip2_peptide_listDF['protein'].str.contains('Reverse')]
    decoy_df = df[df['protein'].str.contains('Reverse')]
    num_decoy_records = decoy_df.index.size
    percent_decoy_records = (num_decoy_records/num_records * 100)
    num_decoy_seqs = len(set(decoy_df['sequence']))
    percent_decoy_seqs = (num_decoy_seqs/num_seqs * 100)

    msg = 'Total number of records {}, sequences {}, decoy records {} ({:.0f}%), decoy sequences {}  ({:.0f}%)'.format(
                                                                                                                num_records, 
                                                                                                                num_seqs, 
                                                                                                                num_decoy_records, 
                                                                                                                percent_decoy_records, 
                                                                                                                num_decoy_seqs, 
                                                                                                                percent_decoy_seqs)
    print(msg)

    df = df[~df['protein'].str.contains('Reverse')]
    df['RatioRank'] = df.AREA_MEDIAN_RATIO_1.rank(ascending=True)

    ax = sns.regplot(x="RatioRank", y="AREA_MEDIAN_RATIO_1", data=df)
    plt.figure()
    
    return df
    




In [ ]:
## Find overlap
def do_ratio_compare_overlap(ip2DF=None,  pdDF=None):
    pdDF = pdDF[~pdDF.ABUNDANCE_RATIOS.isnull()]
    pdDF = pdDF[['Sequence', 'ABUNDANCE_RATIOS']]
    ip2DF = ip2DF[['sequence', 'AREA_MEDIAN_RATIO_1']]
    
    ## Merge on sequence
    df = pd.merge(pdDF, ip2DF, left_on='Sequence', right_on='sequence')
    ## Do simple plot
    sns.regplot('ABUNDANCE_RATIOS','AREA_MEDIAN_RATIO_1', data=df)
    
    return df
    
    
df = do_ratio_compare_overlap(ip2DF=ip2DF, pdDF=pdDF)
df.to_csv('/home/jonesmic/clip/jnk.txt', sep='\t', index=False)
    
df.columns

In [ ]:
pd_file_path = '/usca/asperapoc/NB-CPACT/NB-CPACT-NIBR/NIBR_loaded_UCB_EN80/Analysis/PD2.1/KEA_isoTOP_DN_All.pdResult'
ip2_file_path = '/home/jonesmic/gBuild/jonesmic_github/proteomics-scripts/datanocommit/peptideList.csv'


print('Compare PD on Canonical FASTA vs. IP2')
print('{} vs. {}'.format(pd_file_path, ip2_file_path))
print('load PD')
pdReader = PDReader(pd_result_file=pd_file_path, include_non_quant=False)
pdDF = pdReader.get_target_peptides(include_additional_data=True)
print('Load IP2')
ip2DF = load_ip2_peptide_list(ip2_file_path)


#do_sequence_overlap(ip2DF=ip2DF, pdDF=pdDF)   


In [ ]:
pdDF.head()

In [ ]:
pd_file_path = '/da/dmp/cb/jonesmic/chemgx/data/isoTopAnalysis/jonesmic/pd/pd2.2/KEA_EN80_NomiraFastaSequest_FullTryptic/KEA_EN80.pdResult'
ip2_file_path = '/home/jonesmic/gBuild/jonesmic_github/proteomics-scripts/datanocommit/peptideList.csv'


print('Compare PD on Canonical+Alternative FASTA vs. IP2')
print('{} vs. {}'.format(pd_file_path, ip2_file_path))
print('load PD')
pdReader = PDReader(pd_result_file=pd_file_path, include_non_quant=False)
pdDF = pdReader.get_target_peptides(include_additional_data=True)
print('Load IP2')
ip2DF = load_ip2_peptide_list(ip2_file_path)


#do_sequence_overlap(ip2DF=ip2DF, pdDF=pdDF)   